In [15]:
!pip3 install lxml

In [18]:
!pip install folium
!pip install BeautifulSoup4

     |████████████████████████████████| 115 kB 4.4 MB/s eta 0:00:01


In [19]:

import pandas as pd
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
 

import requests

In [20]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc, features='html.parser')

In [21]:
datadiv=soup.find("table", {"class": "wikitable sortable"})
elementsfull =[]
row=0
for tr in datadiv.findAll("tr"):
    elements=[]
    column=0
    for td in tr.findAll("td"):
        if(td.text!=''):
            elements.append(td.text)
            column+=1
               

    elementsfull.append(elements)        
            
    row+=1

In [22]:
mydf = pd.DataFrame(data=elementsfull)


In [23]:
mydf.columns = ['Postal Code', 'Borough','Neighbourhood']

In [48]:
cols_to_check = ['Postal Code', 'Borough','Neighbourhood']
mydf[cols_to_check] = mydf[cols_to_check].replace({'\n':''}, regex=True)

## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [55]:
cols=[x for i,x in enumerate(mydf.index) if mydf.iat[i,1]=='Not assigned']
mydf= mydf.drop(cols,axis=0)

mydf.reset_index(drop=True,inplace=True)

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [53]:
cols1=[ i for i,x in enumerate(mydf.index) if mydf.iat[i,2] == 'Not assigned']
for i in cols1:
    df1.replace(mydf.iloc[i,2],mydf.iloc[i,1],inplace=True)
    

## number of rows of dataframe.¶

In [54]:
mydf.shape

(102, 3)

In [56]:
mydf.head()

,Postal Code,Borough,Neighbourhood
0,M4A,North York,Victoria Village
1,M5A,Downtown Toronto,"Regent Park, Harbourfront"
2,M6A,North York,"Lawrence Manor, Lawrence Heights"
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
4,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


## Build the dataframe with coordinate

In [57]:
df_cor = pd.read_csv('http://cocl.us/Geospatial_data')
df_cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
df2=pd.merge(mydf,df_cor,how='inner',on=['Postal Code'])
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4A,North York,Victoria Village,43.725882,-79.315572
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
4,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


## Create map of Toronto using latitude and longitude values

In [59]:
map_can = folium.Map(location=[43.7, -79.25], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)
    
map_can

## Cluster those location by K-means using the Latitude and Londtitude

In [60]:
X= df2.values[:,3:]

In [61]:
k_means = KMeans(init = "k-means++", n_clusters = 3, n_init = 10)
k_means.fit(X)
k_means_labels = k_means.labels_
k_means_labels

array([1, 2, 0, 2, 0, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 2, 0, 1,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 0, 1, 2, 0, 2, 2, 0, 0, 1, 2, 0, 2, 2, 0, 0, 1, 2,
       2, 2, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 2, 2, 0, 1, 2, 2, 1, 2, 2, 0,
       0, 1, 2, 2, 0, 0, 1, 2, 2, 0, 2, 2, 0, 0], dtype=int32)

## Add clustering labels

In [62]:
df2.insert(0, 'Cluster Labels', k_means_labels)
df2

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,1,M4A,North York,Victoria Village,43.725882,-79.315572
1,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
2,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
4,0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
5,1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
6,2,M3B,North York,Don Mills,43.745906,-79.352188
7,1,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
8,2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
9,2,M6B,North York,Glencairn,43.709577,-79.445073


In [63]:
# create map
map_clusters = folium.Map(location=[43.7, -79.25], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighbourhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters